# Land Acquisition Analysis - TerraFlow

This notebook provides a complete interface for land acquisition data collection and residual value analysis.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import sys

# Add project root to path for imports
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.append(str(project_root))

from modules.land_acquisition import (
    validate_inputs,
    save_inputs_to_csv,
    get_acquisition_summary
)
from modules.deal_model import LandDeal, LandInputs, create_deal_from_dict
from modules.market_lookup import load_market_data, validate_inputs_against_market, get_market_summary
from utils.scoring import get_color_indicator, format_currency

print("✅ TerraFlow Land Acquisition Module Loaded")
print("📊 Residual value calculation engine ready")

## Market Research

First, let's check market benchmarks for context:

In [ ]:
# Select your market (available: toronto, vancouver, calgary, montreal, default)
MARKET_CITY = 'toronto'

# Load market data
market_data = load_market_data(MARKET_CITY)
market_summary = get_market_summary(MARKET_CITY)

print(f"📍 Market Research: {market_summary['city']}")
print(f"   Land Price Range: {market_summary['land_price_range']} /sqm")
print(f"   Sale Price Range: {market_summary['sale_price_range']} /sqm")
print(f"   Construction Cost: {market_summary['construction_cost_avg']} /sqm")
print(f"   Absorption Rate: {market_summary['absorption_rate']}")
print(f"   Typical Land % GDV: {market_summary['typical_land_gdv']}")
print(f"   Typical Profit Margin: {market_summary['typical_profit']}")
print(f"   Market Strength: {market_summary['market_strength']}")
print(f"   Last Updated: {market_summary['last_updated']}")

## Data Collection

Fill in the land acquisition parameters below:

In [ ]:
# Land acquisition inputs (modify these values)
land_inputs = {
    # Site Information
    'site_name': 'Queen Street West Site',
    'land_area_sqm': 1500.0,
    'asking_price': 750000.0,
    'taxes_fees': 37500.0,
    
    # Zoning & Development
    'zoning': 'Mixed-use Residential',
    'far': 2.0,  # Floor Area Ratio
    'coverage': 0.45,  # Site coverage ratio
    'max_floors': 6,
    'efficiency_ratio': 0.87,  # Net sellable area ratio
    
    # Market Assumptions
    'expected_sale_price_psm': 4200.0,  # Sale price per sqm
    'construction_cost_psm': 2100.0,    # Construction cost per sqm
    'soft_cost_pct': 0.16,              # Soft costs as % of hard costs
    'profit_target_pct': 0.20,          # Developer profit target %
    
    # Additional Costs
    'financing_cost': 45000.0,          # Interest during construction
    'holding_period_months': 30         # Development timeline
}

print("📝 Land acquisition inputs configured.")

In [ ]:
# Validate inputs against market benchmarks
market_warnings = validate_inputs_against_market(land_inputs, MARKET_CITY)

if market_warnings:
    print("⚠️ Market Validation Warnings:")
    for field, warning in market_warnings.items():
        print(f"   {field}: {warning}")
else:
    print("✅ Inputs align with market benchmarks")

# Basic input validation
try:
    validated_inputs = validate_inputs(land_inputs)
    print("✅ Input validation passed")
except ValueError as e:
    print(f"❌ Validation error: {e}")
    raise

## Financial Analysis

Now let's run the complete residual land value analysis:

In [ ]:
# Create and analyze the deal
deal = create_deal_from_dict(land_inputs)

print("🏗️ Development Capacity Analysis:")
print(f"   Gross Buildable Area: {deal.outputs.gross_buildable_sqm:,.0f} sqm")
print(f"   Net Sellable Area: {deal.outputs.net_sellable_sqm:,.0f} sqm")
print(f"   Efficiency Ratio: {deal.inputs.efficiency_ratio:.1%}")
print()

print("💰 Financial Analysis:")
print(f"   Gross Development Value: {format_currency(deal.outputs.gdv)}")
print(f"   Hard Costs: {format_currency(deal.outputs.hard_costs)}")
print(f"   Soft Costs: {format_currency(deal.outputs.soft_costs)}")
print(f"   Total Development Cost: {format_currency(deal.outputs.total_dev_cost)}")
print(f"   Required Profit ({deal.inputs.profit_target_pct:.0%}): {format_currency(deal.outputs.required_profit)}")
print()

print("🎯 Land Valuation:")
print(f"   Residual Land Value: {format_currency(deal.outputs.residual_land_value)}")
print(f"   Asking Price: {format_currency(deal.inputs.asking_price)}")
print(f"   Difference: {format_currency(deal.outputs.asking_vs_residual)}")
print(f"   Land % of GDV: {deal.outputs.land_pct_gdv:.1f}%")
print(f"   Breakeven Sale Price: {format_currency(deal.outputs.breakeven_sale_price)}/sqm")

In [ ]:
# Display viability scores
print("📊 Viability Assessment:")
print(f"   Residual vs Asking: {get_color_indicator(deal.viability.residual_score)} {deal.viability.residual_status}")
print(f"   Land % of GDV: {get_color_indicator(deal.viability.land_pct_score)} {deal.viability.land_pct_status}")
print(f"   Breakeven Risk: {get_color_indicator(deal.viability.breakeven_score)} {deal.viability.breakeven_status}")
print()
print(f"🎯 Overall Assessment: {get_color_indicator(deal.viability.overall_score)} {deal.viability.overall_status}")

# Show sensitivity analysis
print()
print("📈 Sensitivity Analysis:")
print(f"   Base Case: {format_currency(deal.sensitivity.base_residual)}")
print(f"   Sales -10%: {format_currency(deal.sensitivity.sales_down_10pct)} ({format_currency(deal.sensitivity.sales_impact)} impact)")
print(f"   Costs +10%: {format_currency(deal.sensitivity.costs_up_10pct)} ({format_currency(deal.sensitivity.costs_impact)} impact)")

## Data Export

Save the analysis results for dashboard and comparison:

In [ ]:
# Save to CSV for tracking and dashboard use
try:
    # Convert deal to DataFrame row
    deal_row = {**land_inputs}
    deal_row.update({
        'residual_land_value': deal.outputs.residual_land_value,
        'land_pct_gdv': deal.outputs.land_pct_gdv,
        'breakeven_sale_price': deal.outputs.breakeven_sale_price,
        'overall_score': deal.viability.overall_score,
        'overall_status': deal.viability.overall_status,
        'gdv': deal.outputs.gdv,
        'total_dev_cost': deal.outputs.total_dev_cost,
        'analysis_date': pd.Timestamp.now().strftime('%Y-%m-%d'),
        'market_city': MARKET_CITY
    })
    
    df_new = pd.DataFrame([deal_row])
    
    # Append to existing data
    csv_path = project_root / 'data' / 'processed' / 'acquisitions.csv'
    
    if csv_path.exists():
        df_existing = pd.read_csv(csv_path)
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    else:
        df_combined = df_new
    
    df_combined.to_csv(csv_path, index=False)
    print(f"📁 Analysis saved to: {csv_path}")
    print(f"📊 Total deals in database: {len(df_combined)}")
    
except Exception as e:
    print(f"❌ Save error: {e}")

## Summary Table

Quick summary for comparison with other sites:

In [ ]:
# Create summary table
summary_data = {
    'Metric': [
        'Site Name',
        'Land Area (sqm)',
        'Asking Price',
        'Price per sqm',
        'Residual Land Value',
        'Land % of GDV',
        'Breakeven Price/sqm',
        'Overall Score'
    ],
    'Value': [
        deal.inputs.site_name,
        f"{deal.inputs.land_area_sqm:,.0f}",
        format_currency(deal.inputs.asking_price),
        format_currency(deal.outputs.land_psm),
        format_currency(deal.outputs.residual_land_value),
        f"{deal.outputs.land_pct_gdv:.1f}%",
        format_currency(deal.outputs.breakeven_sale_price),
        f"{get_color_indicator(deal.viability.overall_score)} {deal.viability.overall_status}"
    ]
}

summary_df = pd.DataFrame(summary_data)
display(summary_df)

## Next Steps

1. **Dashboard Review**: View results in Streamlit dashboard for visual analysis
2. **Sensitivity Testing**: Adjust key assumptions and re-run analysis
3. **Market Comparison**: Compare with other sites in the database
4. **Risk Assessment**: Review warnings and flags for decision making

The complete analysis is now saved and ready for dashboard visualization and further comparison analysis.